# Fraud Detection Models

This notebook aims to create a prediction of fraudulent purchasing in an e-commerce through both supervised and unsupervised machine learning modeling using data composed of devices and IP addresses, along with demographics used in both fraudulent and legitimate purchases.

In [1]:
# importing necessary libraries and modules

import numpy as np
import pandas as pd
import itertools

from scipy import special
from datetime import timedelta
from IPython.display import display

In [2]:
# reading necessary data
fraud_df = pd.read_csv('Fraud_Data.csv',header=0)
ISP_df = pd.read_csv('IpAddress_to_Country.csv',header=0)

In [3]:
# checking out the data types for the fraud dataset
fraud_df.dtypes

id                  int64
cadastro           object
compra             object
valor               int64
id_dispositivo     object
fonte              object
browser            object
genero             object
idade               int64
ip                float64
fraude              int64
dtype: object

In [4]:
# checking for null values for the fraud dataset
for col in fraud_df.columns:
    print('Null values in {}: {}'.format(col, fraud_df[col].isnull().sum()))

Null values in id: 0
Null values in cadastro: 0
Null values in compra: 0
Null values in valor: 0
Null values in id_dispositivo: 0
Null values in fonte: 0
Null values in browser: 0
Null values in genero: 0
Null values in idade: 0
Null values in ip: 0
Null values in fraude: 0


In [5]:
# describing the fraud dataset for the fraud dataset
fraud_df.describe()

,id,valor,idade,ip,fraude
count,151112.000000,151112.000000,151112.000000,1.511120e+05,151112.000000
mean,200171.040970,36.935372,33.140704,2.152145e+09,0.093646
std,115369.285024,18.322762,8.617733,1.248497e+09,0.291336
min,2.000000,9.000000,18.000000,5.209350e+04,0.000000
25%,100642.500000,22.000000,27.000000,1.085934e+09,0.000000
50%,199958.000000,35.000000,33.000000,2.154770e+09,0.000000
75%,300054.000000,49.000000,39.000000,3.243258e+09,0.000000
max,400000.000000,154.000000,76.000000,4.294850e+09,1.000000


In [6]:
# checking out the data types for the IP dataset
ISP_df.dtypes

limite_inferior_ip    float64
limite_superior_ip      int64
pais                   object
dtype: object

In [7]:
ISP_df.head()

,limite_inferior_ip,limite_superior_ip,pais
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [8]:
# upper and lower IP limits are in different data types.
# checking whether that was merely a dot by the end.

ip_floats = ISP_df.limite_inferior_ip.apply(lambda x: not x.is_integer()).sum()
print('Non-zero decimals in lower IP limits: {}'.format(ip_floats))

Non-zero decimals in lower IP limits: 0


In [9]:
# turning upper IP limit to a float64 so there'll be no errors in comparing the two datasets
ISP_df.limite_superior_ip = ISP_df.limite_superior_ip.astype(np.float64)
ISP_df.dtypes

limite_inferior_ip    float64
limite_superior_ip    float64
pais                   object
dtype: object

In [10]:
# checking for null values for the IP dataset
for col in ISP_df.columns:
    print('Null values in {}: {}'.format(col, ISP_df[col].isnull().sum()))

Null values in limite_inferior_ip: 0
Null values in limite_superior_ip: 0
Null values in pais: 0


In [11]:
# describing the fraud dataset for the IP dataset
ISP_df.describe()

,limite_inferior_ip,limite_superior_ip
count,1.388460e+05,1.388460e+05
mean,2.724532e+09,2.724557e+09
std,8.975215e+08,8.974979e+08
min,1.677722e+07,1.677747e+07
25%,1.919930e+09,1.920008e+09
50%,3.230887e+09,3.230888e+09
75%,3.350465e+09,3.350466e+09
max,3.758096e+09,3.758096e+09


In [12]:
# strange value at 75% percentile for the lower limit,
# checking whether it is merely a display issue

display(ISP_df[ISP_df.limite_inferior_ip == ISP_df.limite_inferior_ip.max()])
display(ISP_df[ISP_df.limite_superior_ip == ISP_df.limite_superior_ip.max()])

,limite_inferior_ip,limite_superior_ip,pais
138845,3.758096e+09,3.758096e+09,Australia


,limite_inferior_ip,limite_superior_ip,pais
138845,3.758096e+09,3.758096e+09,Australia


In [ ]:
# defining which country an IP belongs to
def ip_to_country(ip):
    tmp = ISP_df[(ISP_df.limite_inferior_ip <= ip)
            & (ISP_df.limite_superior_ip >= ip)].pais
    if tmp.shape[0] == 1:
        return tmp.iloc[0]

fraud_df["pais"] = fraud_df.ip.apply(ip_to_country)
fraud_df.head()

In [ ]:
fraud_df.to_csv("fraud_data_country.csv", index=False)

# Exploratory Data Analysis

In [ ]:
# importing necessary libraries

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# recreating the fraud dataset with country data

fraud_df = pd.read_csv("fraud_data_country.csv", header=0)
fraud_df.head()